In [28]:
import $file.^.Magic

import $file.$

In [29]:
val baseUrl = "https://f001.backblazeb2.com/file/LLMs-from-scratch/gpt2/124M" // backup
// val baseUrl = "https://openaipublic.blob.core.windows.net/gpt-2/models/124M" // backup
val hparamsFilename = "hparams.json"
val filenames = List("checkpoint", "encoder.json", hparamsFilename, "model.ckpt.data-00000-of-00001", "model.ckpt.index", "model.ckpt.meta", "vocab.bpe")

val outputDir = "data/openai124M"

baseUrl: String = "https://f001.backblazeb2.com/file/LLMs-from-scratch/gpt2/124M"
hparamsFilename: String = "hparams.json"
filenames: List[String] = List(
  "checkpoint",
  "encoder.json",
  "hparams.json",
  "model.ckpt.data-00000-of-00001",
  "model.ckpt.index",
  "model.ckpt.meta",
  "vocab.bpe"
)
outputDir: String = "data/openai124M"

In [8]:
filenames.foreach { filename =>
  println(s"Downloading $filename...")
  Magic.!("curl", "--create-dirs", "-O", "--output-dir", outputDir, s"$baseUrl/$filename")
}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    77  100    77    0     0    107      0 --:--:-- --:--:-- --:--:--   107


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  3 1017k    3 40522    0     0  35490      0  0:00:29  0:00:01  0:00:28 35483
100 1017k  100 1017k    0     0   514k      0  0:00:01  0:00:01 --:--:--  514k


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    90  100    90    0     0    138      0 --:--:-- --:--:-- --:--:--   138


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  474M    0  263k    0     0   180k      0  0:44:54  0:00:01  0:44:53  180k
  1  474M    1 8479k    0     0  3442k      0  0:02:21  0:00:02  0:02:19 3441k
  5  474M    5 23.9M    0     0  7066k      0  0:01:08  0:00:03  0:01:05 7065k
  8  474M    8 39.2M    0     0  8971k      0  0:00:54  0:00:04  0:00:50 8969k
 11  474M   11 55.7M    0     0  10.0M      0  0:00:47  0:00:05  0:00:42 11.2M
 14  474M   14 70.8M    0     0  10.5M      0  0:00:44  0:00:06  0:00:38 13.4M
 17  474M   17 82.2M    0     0  11.0M      0  0:00:43  0:00:07  0:00:36 14.8M
 20  474M   20 97.3M    0     0  11.4M      0  0:00:41  0:00:08  0:00:33 14.6M
 23  474M   23  112M    0     0  11.9M      0  0:00

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  5215  100  5215    0     0   7743      0 --:--:-- --:--:-- --:--:--  7737
100  5215  100  5215    0     0   7742      0 --:--:-- --:--:-- --:--:--  7737


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  8  460k    8 40512    0     0  42520      0  0:00:11 --:--:--  0:00:11 42509
100  460k  100  460k    0     0   290k      0  0:00:01  0:00:01 --:--:--  291k


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 42  445k   42  191k    0     0   141k      0  0:00:03  0:00:01  0:00:02  141k
100  445k  100  445k    0     0   282k      0  0:00:01  0:00:01 --:--:--  282k


In [30]:
Magic.!("pip", "install", "tensorflow==2.16.*")


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [31]:
import $ivy.`com.lihaoyi::ujson:4.1.0`

import scala.io.Source

val hparamsMap = ujson.read(Source.fromFile(s"$outputDir/$hparamsFilename").mkString)

case class GPTConfig(
  vocabularySize: Int,
  contextLength: Int,
  embeddingDimension: Int,
  attentionHeadsCount: Int,
  layersCount: Int,
  dropoutRate: Double,
  queryKeyValueBias: Boolean
)

val gptConfig = GPTConfig(
  vocabularySize = hparamsMap("n_vocab").num.toInt,
  contextLength = hparamsMap("n_ctx").num.toInt,
  embeddingDimension = hparamsMap("n_embd").num.toInt,
  attentionHeadsCount = hparamsMap("n_head").num.toInt,
  layersCount = hparamsMap("n_layer").num.toInt,
  dropoutRate = 0.1,
  queryKeyValueBias = true
)

import $ivy.$
import scala.io.Source
hparamsMap: ujson.Value.Value = Obj(
  value = Map(
    "n_vocab" -> Num(value = 50257.0),
    "n_ctx" -> Num(value = 1024.0),
    "n_embd" -> Num(value = 768.0),
    "n_head" -> Num(value = 12.0),
    "n_layer" -> Num(value = 12.0)
  )
)
defined class GPTConfig
gptConfig: GPTConfig = GPTConfig(
  vocabularySize = 50257,
  contextLength = 1024,
  embeddingDimension = 768,
  attentionHeadsCount = 12,
  layersCount = 12,
  dropoutRate = 0.1,
  queryKeyValueBias = true
)

In [32]:
import $ivy.`dev.scalapy::scalapy-core:0.5.3`

import me.shadaj.scalapy.py

val tf = py.module("tensorflow")
val np = py.module("numpy")

import $ivy.$
import me.shadaj.scalapy.py
tf: py.Module = <module 'tensorflow' from '/usr/local/lib/python3.12/site-packages/tensorflow/__init__.py'>
np: py.Module = <module 'numpy' from '/usr/local/lib/python3.12/site-packages/numpy/__init__.py'>

In [33]:
val checkpoint = tf.train.latest_checkpoint(outputDir)
val variableNames = tf.train.list_variables(checkpoint).as[Seq[(String, Seq[Int])]].map { 
  case (variableName, _) => variableName 
}.toList
variableNames.sorted.foreach(println)

model/h0/attn/c_attn/b
model/h0/attn/c_attn/w
model/h0/attn/c_proj/b
model/h0/attn/c_proj/w
model/h0/ln_1/b
model/h0/ln_1/g
model/h0/ln_2/b
model/h0/ln_2/g
model/h0/mlp/c_fc/b
model/h0/mlp/c_fc/w
model/h0/mlp/c_proj/b
model/h0/mlp/c_proj/w
model/h1/attn/c_attn/b
model/h1/attn/c_attn/w
model/h1/attn/c_proj/b
model/h1/attn/c_proj/w
model/h1/ln_1/b
model/h1/ln_1/g
model/h1/ln_2/b
model/h1/ln_2/g
model/h1/mlp/c_fc/b
model/h1/mlp/c_fc/w
model/h1/mlp/c_proj/b
model/h1/mlp/c_proj/w
model/h10/attn/c_attn/b
model/h10/attn/c_attn/w
model/h10/attn/c_proj/b
model/h10/attn/c_proj/w
model/h10/ln_1/b
model/h10/ln_1/g
model/h10/ln_2/b
model/h10/ln_2/g
model/h10/mlp/c_fc/b
model/h10/mlp/c_fc/w
model/h10/mlp/c_proj/b
model/h10/mlp/c_proj/w
model/h11/attn/c_attn/b
model/h11/attn/c_attn/w
model/h11/attn/c_proj/b
model/h11/attn/c_proj/w
model/h11/ln_1/b
model/h11/ln_1/g
model/h11/ln_2/b
model/h11/ln_2/g
model/h11/mlp/c_fc/b
model/h11/mlp/c_fc/w
model/h11/mlp/c_proj/b
model/h11/mlp/c_proj/w
model/h2/attn/c_

checkpoint: py.Dynamic = data/openai124M/model.ckpt
variableNames: List[String] = List(
  "model/h0/attn/c_attn/b",
  "model/h0/attn/c_attn/w",
  "model/h0/attn/c_proj/b",
  "model/h0/attn/c_proj/w",
  "model/h0/ln_1/b",
  "model/h0/ln_1/g",
  "model/h0/ln_2/b",
  "model/h0/ln_2/g",
  "model/h0/mlp/c_fc/b",
  "model/h0/mlp/c_fc/w",
  "model/h0/mlp/c_proj/b",
  "model/h0/mlp/c_proj/w",
  "model/h1/attn/c_attn/b",
  "model/h1/attn/c_attn/w",
  "model/h1/attn/c_proj/b",
  "model/h1/attn/c_proj/w",
  "model/h1/ln_1/b",
  "model/h1/ln_1/g",
  "model/h1/ln_2/b",
  "model/h1/ln_2/g",
  "model/h1/mlp/c_fc/b",
  "model/h1/mlp/c_fc/w",
  "model/h1/mlp/c_proj/b",
  "model/h1/mlp/c_proj/w",
  "model/h10/attn/c_attn/b",
  "model/h10/attn/c_attn/w",
  "model/h10/attn/c_proj/b",
  "model/h10/attn/c_proj/w",
  "model/h10/ln_1/b",
  "model/h10/ln_1/g",
  "model/h10/ln_2/b",
  "model/h10/ln_2/g",
  "model/h10/mlp/c_fc/b",
  "model/h10/mlp/c_fc/w",
  "model/h10/mlp/c_proj/b",
  "model/h10/mlp/c_proj/w",


In [34]:
val torch = py.module("torch")

torch: py.Module = <module 'torch' from '/usr/local/lib/python3.12/site-packages/torch/__init__.py'>

In [35]:
type Model = py.Dynamic
type NpArray = py.Dynamic

def toTorchParameter(npArray: NpArray) =
  torch.nn.Parameter(torch.tensor(npArray))

def loadModelWeights(model: Model): Unit =
  variableNames.foreach { variableName =>
    val variableValue = np.squeeze(tf.train.load_variable(checkpoint, variableName))
    variableName.split("/").drop(1).toList match {
      case s"h$transformerBlockIndexString" :: tail =>
        val transformerBlockIndex = transformerBlockIndexString.toInt
        tail match {
          case "attn" :: tail =>
            tail match {
              case "c_attn" :: tail =>
                val Seq(queryVariableValue, keyVariableValue, valueVariableValue) = np.split(variableValue, 3, axis = -1).as[Seq[NpArray]]
                tail match {
                  case "b" :: _ => 
                    model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).multiHeadAttention.weightsQuery.bias = toTorchParameter(queryVariableValue)
                    model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).multiHeadAttention.weightsKey.bias = toTorchParameter(keyVariableValue)
                    model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).multiHeadAttention.weightsValue.bias = toTorchParameter(valueVariableValue)
                  case "w" :: _ => 
                    model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).multiHeadAttention.weightsQuery.weight = toTorchParameter(queryVariableValue.T)
                    model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).multiHeadAttention.weightsKey.weight = toTorchParameter(keyVariableValue.T)
                    model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).multiHeadAttention.weightsValue.weight = toTorchParameter(valueVariableValue.T)
                }
              case "c_proj" :: tail =>
                tail match {
                  case "b" :: _ => model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).multiHeadAttention.outputProjection.bias = toTorchParameter(variableValue)
                  case "w" :: _ => model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).multiHeadAttention.outputProjection.weight = toTorchParameter(variableValue.T)
                }
            }
          case "ln_1" :: tail =>
            val torchParameter = toTorchParameter(variableValue)
            tail match {
              case "b" :: _ => model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).normalization1.shift = torchParameter
              case "g" :: _ => model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).normalization1.scale = torchParameter
            }
          case "ln_2" :: tail =>
            val torchParameter = toTorchParameter(variableValue)
            tail match {
              case "b" :: _ => model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).normalization2.shift = torchParameter
              case "g" :: _ => model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).normalization2.scale = torchParameter
            }
          case "mlp" :: tail =>
            tail match {
              case "c_fc" :: tail =>
                tail match {
                  case "b" :: _ => model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).feedForward.layers.bracketAccess(0).bias = toTorchParameter(variableValue)
                  case "w" :: _ => model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).feedForward.layers.bracketAccess(0).weight = toTorchParameter(variableValue.T)
                }
              case "c_proj" :: tail =>
                tail match {
                  case "b" :: _ => model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).feedForward.layers.bracketAccess(2).bias = toTorchParameter(variableValue)
                  case "w" :: _ => model.transformerBlocksLayer.bracketAccess(transformerBlockIndex).feedForward.layers.bracketAccess(2).weight = toTorchParameter(variableValue.T)
                }
            }
        }
      case "ln_f" :: tail =>
        val torchParameter = toTorchParameter(variableValue)
        tail match {
          case "b" :: _ => model.finalNormalizationLayer.shift = torchParameter
          case "g" :: _ => model.finalNormalizationLayer.scale = torchParameter
        }
      case "wpe" :: _ => model.positionEmbeddingLayer.weight = toTorchParameter(variableValue)
      case "wte" :: _ => 
        val torchParameter = toTorchParameter(variableValue)
        model.tokenEmbeddingLayer.weight = torchParameter
        model.outputLayer.weight = torchParameter
    }
  }

cmd35.sc:18: match may not be exhaustive.
It would fail on the following inputs: List((x: String forSome x not in ("b", "w"))), Nil
                tail match {
                ^
cmd35.sc:29: match may not be exhaustive.
It would fail on the following inputs: List((x: String forSome x not in ("b", "w"))), Nil
                tail match {
                ^
cmd35.sc:15: match may not be exhaustive.
It would fail on the following inputs: List((x: String forSome x not in ("c_attn", "c_proj"))), Nil
            tail match {
            ^
cmd35.sc:36: match may not be exhaustive.
It would fail on the following inputs: List((x: String forSome x not in ("b", "g"))), Nil
            tail match {
            ^
cmd35.sc:42: match may not be exhaustive.
It would fail on the following inputs: List((x: String forSome x not in ("b", "g"))), Nil
            tail match {
            ^
cmd35.sc:49: match may not be exhaustive.
It would fail on the following inputs: List((x: String forSome x not in ("b",

defined type Model
defined type NpArray
defined function toTorchParameter
defined function loadModelWeights

In [36]:
Magic.!("pip", "install", "torch==2.4.*")


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [37]:
val torch = py.module("torch")

torch: py.Module = <module 'torch' from '/usr/local/lib/python3.12/site-packages/torch/__init__.py'>

In [38]:
import py.PyQuote

type TorchTensor = py.Dynamic

// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class MultiHeadAttention(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
def MultiHeadAttention(
  inputDimension: Int,
  outputDimension: Int,
  dropoutProbability: Double,
  contextLength: Int,
  headsCount: Int,
  queryKeyValueBias: Boolean
): py.Dynamic = {
  assert(outputDimension % headsCount == 0, "Output dimension must be a multiple of heads count")
  val headDimension = outputDimension / headsCount
    
  val init = (self: py.Dynamic) => {
    self.weightsQuery = torch.nn.Linear(inputDimension, outputDimension, bias = queryKeyValueBias)
    self.weightsKey = torch.nn.Linear(inputDimension, outputDimension, bias = queryKeyValueBias)
    self.weightsValue = torch.nn.Linear(inputDimension, outputDimension, bias = queryKeyValueBias)
    self.outputProjection = torch.nn.Linear(outputDimension, outputDimension)
    self.dropout = torch.nn.Dropout(dropoutProbability)
    self.register_buffer("mask", torch.triu(torch.ones(contextLength, contextLength), diagonal = 1))
      
    val forward = (batchedInputs: TorchTensor) => {
      val (batchesCount, tokensCount, tokenDimension) = batchedInputs.shape.as[(Int, Int, Int)]
      val queries = self.weightsQuery(batchedInputs)
        .view(batchesCount, tokensCount, headsCount, headDimension)
        .transpose(1, 2)
      val keys = self.weightsKey(batchedInputs)
        .view(batchesCount, tokensCount, headsCount, headDimension)
        .transpose(1, 2)
      val values = self.weightsValue(batchedInputs)
        .view(batchesCount, tokensCount, headsCount, headDimension)
        .transpose(1, 2)
      val attentionScores = py"$queries @ $keys.transpose(2, 3)"
      attentionScores.masked_fill_(py"${self.mask}.bool()[:$tokensCount, :$tokensCount]", -torch.inf)
      val attentionWeights = self.dropout(torch.softmax(py"$attentionScores / $headDimension**0.5", dim = -1))
      self.outputProjection(
        py"$attentionWeights @ $values"
          .transpose(1, 2)
          .reshape(batchesCount, tokensCount, outputDimension)
      )
    }
    self.forward = forward
  }
  py.Dynamic.global.MultiHeadAttention(init)
}

import py.PyQuote
defined type TorchTensor
defined function MultiHeadAttention

In [39]:
// Workaround to define a class that inherits from a Python class
// Because it mostly uses Python operators, it's implemented fully in Python
py.exec {
  s"""import torch
     |import torch.nn as nn
     |
     |class GELU(nn.Module):
     |  def __init__(self):
     |    super().__init__()
     |
     |  def forward(self, inputs):
     |    return 0.5 * inputs * (
     |      1 + torch.tanh(
     |        torch.sqrt(torch.tensor(2.0 / torch.pi)) * (inputs + 0.044715 * torch.pow(inputs, 3))
     |      )
     |    )
     |""".stripMargin
}
def GELU() = py.Dynamic.global.GELU()

defined function GELU

In [40]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class FeedForward(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
def FeedForward(
  embeddingDimension: Int
): py.Dynamic = {
  val init = (self: py.Dynamic) => {
    self.layers = torch.nn.Sequential(
      torch.nn.Linear(embeddingDimension, 4 * embeddingDimension),
      GELU(),
      torch.nn.Linear(4 * embeddingDimension, embeddingDimension)
    )
      
    val forward = (inputs: TorchTensor) => self.layers(inputs)
    self.forward = forward
  }
  py.Dynamic.global.FeedForward(init)
}

defined function FeedForward

In [41]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class NormalizationLayer(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
def NormalizationLayer(
  embeddingDimension: Int
): py.Dynamic = {
  val epsilon = 1e-5
  val init = (self: py.Dynamic) => {
    self.scale = torch.nn.Parameter(torch.ones(embeddingDimension))
    self.shift = torch.nn.Parameter(torch.zeros(embeddingDimension))
      
    val forward = (inputs: TorchTensor) => {
      val mean = inputs.mean(dim = -1, keepdim = true)
      val variance = inputs.`var`(dim = -1, keepdim = true, unbiased = false)
      val normalizedInputs = py"($inputs - $mean) / torch.sqrt($variance + $epsilon)"
      py"${self.scale} * $normalizedInputs + ${self.shift}"
    }
    self.forward = forward
  }
  py.Dynamic.global.NormalizationLayer(init)
}

defined function NormalizationLayer

In [42]:
import scala.util.chaining._

py.exec {
  s"""import torch.nn as nn
     |
     |class TransformerBlock(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
def TransformerBlock(
  config: GPTConfig
): py.Dynamic = {
  val init = (self: py.Dynamic) => {
    self.multiHeadAttention = MultiHeadAttention(
      inputDimension = config.embeddingDimension,
      outputDimension = config.embeddingDimension,
      dropoutProbability = config.dropoutRate,
      contextLength = config.contextLength,
      headsCount = config.attentionHeadsCount,
      queryKeyValueBias = config.queryKeyValueBias
    )
    self.feedForward = FeedForward(config.embeddingDimension)
    self.normalization1 = NormalizationLayer(config.embeddingDimension)
    self.normalization2 = NormalizationLayer(config.embeddingDimension)
    self.dropoutShortcut = torch.nn.Dropout(config.dropoutRate)
    
    val forward = (inputs: TorchTensor) => {
      val shortcut = inputs
      val newShortcut = inputs
        .pipe(self.normalization1(_))
        .pipe(self.multiHeadAttention(_))
        .pipe(self.dropoutShortcut(_))
        .pipe(o => py"$o + $shortcut")
      newShortcut
        .pipe(self.normalization2(_))
        .pipe(self.feedForward(_))
        .pipe(self.dropoutShortcut(_))
        .pipe(o => py"$o + $newShortcut")
    }
    self.forward = forward
  }
  py.Dynamic.global.TransformerBlock(init)
}

import scala.util.chaining._
defined function TransformerBlock

In [43]:
// Workaround to define a class that inherits from a Python class
py.exec {
  s"""import torch.nn as nn
     |
     |class GPTModel(nn.Module):
     |  def __init__(self, init):
     |    super().__init__()
     |    init(self)
     |""".stripMargin
}
def GPTModel(
  config: GPTConfig
): Model = {
  val transformerBlocks = Seq.fill(config.layersCount)(TransformerBlock(config))
  val init = (self: py.Dynamic) => {
    self.tokenEmbeddingLayer = torch.nn.Embedding(config.vocabularySize, config.embeddingDimension)
    self.positionEmbeddingLayer = torch.nn.Embedding(config.contextLength, config.embeddingDimension)
    self.dropoutEmbeddingLayer = torch.nn.Dropout(config.dropoutRate)
    self.transformerBlocksLayer = py"nn.Sequential(*${transformerBlocks.toPythonProxy})"
    self.finalNormalizationLayer = NormalizationLayer(config.embeddingDimension)
    self.outputLayer = torch.nn.Linear(config.embeddingDimension, config.vocabularySize, bias = false)
      
    val forward = (batchedInputs: TorchTensor) => {
      val (_, sequenceLength) = batchedInputs.shape.as[(Int, Int)]
      val tokenEmbeddings = self.tokenEmbeddingLayer(batchedInputs)
      val positionEmbeddings = self.positionEmbeddingLayer(torch.arange(sequenceLength, device = batchedInputs.device))
      py"$tokenEmbeddings + $positionEmbeddings"
        .pipe(self.dropoutEmbeddingLayer(_))
        .pipe(self.transformerBlocksLayer(_))
        .pipe(self.finalNormalizationLayer(_))
        .pipe(self.outputLayer(_))
    }
    self.forward = forward
  }
  py.Dynamic.global.GPTModel(init)
}

defined function GPTModel

In [44]:
val model = GPTModel(gptConfig)
loadModelWeights(model)
val device = torch.device(if (torch.cuda.is_available().as[Boolean]) "cuda" else "cpu")
model.to(device)
model.eval()

model: Model = GPTModel(
  (tokenEmbeddingLayer): Embedding(50257, 768)
  (positionEmbeddingLayer): Embedding(1024, 768)
  (dropoutEmbeddingLayer): Dropout(p=0.1, inplace=False)
  (transformerBlocksLayer): Sequential(
    (0): TransformerBlock(
      (multiHeadAttention): MultiHeadAttention(
        (weightsQuery): Linear(in_features=768, out_features=768, bias=True)
        (weightsKey): Linear(in_features=768, out_features=768, bias=True)
        (weightsValue): Linear(in_features=768, out_features=768, bias=True)
        (outputProjection): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (feedForward): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (normalization1): NormalizationLayer()
      (normalization2): NormalizationLayer()
      (d

In [45]:
import py.SeqConverters

type Tokenizer = py.Dynamic

def textToTokenIds(
  text: String, 
  tokenizer: Tokenizer
): TorchTensor = {
  val allowedSpecial = py.Dynamic.global.set(Seq("<|endoftext|>").toPythonProxy)
  val encodedText = tokenizer.encode(text, allowed_special = allowedSpecial)
  torch.tensor(encodedText).unsqueeze(0)
}
    
def tokenIdsToText(
  tokenIds: TorchTensor, 
  tokenizer: Tokenizer
): String =
  tokenizer.decode(tokenIds.squeeze(0).tolist()).as[String]

def generateText(
  model: Model,
  maxNewTokens: Int,
  contextLength: Int,
  temperature: Double,
  topK: Option[Int] = None
)(
  encodedInput: TorchTensor
): TorchTensor =
  LazyList.iterate(encodedInput) { currentEncodedOutput =>
    val croppedInput = py"$currentEncodedOutput[:, -$contextLength:]"
    val logits = py.`with`(torch.no_grad()) { _ =>
      model(croppedInput)
    }
    py"$logits[:, -1, :]"
      .pipe { logits =>
        topK match {
          case Some(topK) =>
            val (topLogits, _) = torch.topk(logits, topK).as[(TorchTensor, TorchTensor)]
            val minValue = py"$topLogits[:, -1]"
            torch.where(
              py"$logits < $minValue",
              torch.tensor(-torch.inf).to(logits.device),
              logits
            )
          case None => logits
        }
      }
      .pipe { logits =>
        if (temperature > 0)
          logits
            .pipe(logits => py"$logits / $temperature")
            .pipe(torch.softmax(_, dim = -1))
            .pipe(torch.multinomial(_, num_samples = 1))
        else 
          logits
            .pipe(torch.argmax(_, dim = -1, keepdim = true))
      }
      .pipe(nextEncodedOutput => torch.cat((currentEncodedOutput, nextEncodedOutput), dim = 1))
  }.drop(maxNewTokens).head

import py.SeqConverters
defined type Tokenizer
defined function textToTokenIds
defined function tokenIdsToText
defined function generateText

In [46]:
Magic.!("pip", "install", "tiktoken==0.7.*")


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [47]:
val tiktoken = py.module("tiktoken")

tiktoken: py.Module = <module 'tiktoken' from '/usr/local/lib/python3.12/site-packages/tiktoken/__init__.py'>

In [48]:
torch.manual_seed(123)
val exampleText = "Every effort moves you"
val tokenizer = tiktoken.get_encoding("gpt2")
val outputTextIds = generateText(
  model = model, 
  maxNewTokens = 25,
  contextLength = gptConfig.contextLength,
  temperature = 1.5,
  topK = Some(50)
)(
  encodedInput = textToTokenIds(exampleText, tokenizer)
)
val decodedOutputText = tokenIdsToText(outputTextIds, tokenizer)
println(s"Output text: $decodedOutputText")

Output text: Every effort moves you toward finding an ideal new way to practice something!

What makes us want to be on top of that?




res48_0: py.Dynamic = <torch._C.Generator object at 0xffff28df8fd0>
exampleText: String = "Every effort moves you"
tokenizer: py.Dynamic = <Encoding 'gpt2'>
outputTextIds: TorchTensor = tensor([[6109, 3626, 6100,  345, 3812, 4917,  281, 7306,  649,  835,  284, 3357,
         1223,    0,  198,  198, 2061, 1838,  514,  765,  284,  307,  319, 1353,
          286,  326,   30,  198,  198]])
decodedOutputText: String = """Every effort moves you toward finding an ideal new way to practice something!

What makes us want to be on top of that?

"""